In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

data = pd.read_csv("healthcare-dataset-stroke-data.csv")

balanced_data = data.dropna()
balanced_data = balanced_data.drop(columns= "id")
balanced_data = balanced_data[balanced_data.age > 20]
balanced_data = balanced_data.drop(balanced_data.index[balanced_data["gender"] == "Other"])
balanced_data = balanced_data.drop(balanced_data.index[balanced_data["work_type"] == "Never_worked"])

features = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', "bmi", "smoking_status"]
cat_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'hypertension', 'heart_disease']
num_columns = ['age', 'avg_glucose_level', 'bmi']

bal_data_train, bal_data_test = train_test_split(balanced_data, train_size = 0.8, random_state=1, stratify= balanced_data["stroke"])
bal_data_train, bal_data_val = train_test_split(bal_data_train, train_size = 0.8, random_state=1)

X_train = bal_data_train.drop('stroke', axis = 1)
y_train = bal_data_train['stroke']

X_val = bal_data_val.drop('stroke', axis = 1)
y_val = bal_data_val['stroke']

X_test = bal_data_test.drop('stroke', axis = 1)
y_test = bal_data_test['stroke']

In [2]:
preprocessing = ColumnTransformer([
    ('one-hot-encoder', OneHotEncoder(sparse_output=False), cat_columns),
    ('scaler', StandardScaler(), num_columns)
])

In [3]:
X_train2 = preprocessing.fit_transform(X_train)
X_val2 = preprocessing.transform(X_val)
X_test2 = preprocessing.transform(X_test)

Initial model starts at 0.07 recall

In [4]:
import tensorflow as tf

tf.random.set_seed(1)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=20))
model.add(tf.keras.layers.Dense(512, 'relu'))
model.add(tf.keras.layers.Dense(512, 'relu'))
model.add(tf.keras.layers.Dense(1, "sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['Recall'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               10752     
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 1)                 513       
                                                                 
Total params: 273,921
Trainable params: 273,921
Non-trainable params: 0
_________________________________________________________________


In [5]:
model_history = model.fit(X_train2, y_train, validation_data=(X_val2, y_val), epochs=10)

Epoch 1/10
79/79 [==============================] - 1s 7ms/step - loss: 0.1976 - recall: 0.0078 - val_loss: 0.2033 - val_recall: 0.0000e+00
Epoch 2/10
79/79 [==============================] - 0s 5ms/step - loss: 0.1696 - recall: 0.0000e+00 - val_loss: 0.1989 - val_recall: 0.0000e+00
Epoch 3/10
79/79 [==============================] - 0s 5ms/step - loss: 0.1656 - recall: 0.0000e+00 - val_loss: 0.1991 - val_recall: 0.0000e+00
Epoch 4/10
79/79 [==============================] - 0s 4ms/step - loss: 0.1591 - recall: 0.0000e+00 - val_loss: 0.2015 - val_recall: 0.0000e+00
Epoch 5/10
79/79 [==============================] - 0s 5ms/step - loss: 0.1568 - recall: 0.0000e+00 - val_loss: 0.2074 - val_recall: 0.0263
Epoch 6/10
79/79 [==============================] - 0s 4ms/step - loss: 0.1507 - recall: 0.0234 - val_loss: 0.2152 - val_recall: 0.0000e+00
Epoch 7/10
79/79 [==============================] - 0s 4ms/step - loss: 0.1473 - recall: 0.0469 - val_loss: 0.2280 - val_recall: 0.0789
Epoch 8/10
7

In [6]:
model.evaluate(X_test2, y_test)

25/25 [==============================] - 0s 2ms/step - loss: 0.2334 - recall: 0.0238


[0.2334386706352234, 0.02380952425301075]

https://imbalanced-learn.org/stable/miscellaneous.html

https://www.freecodecamp.org/news/binary-classification-made-simple-with-tensorflow/


In [7]:
tf.test.is_built_with_cuda()

True

In [8]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]